## Business understanding

### What decisions need to be made?
The decision to be made is which city in Wyoming should be chosen for the next Pawdacity store.
The criteria are:
- The new store must be located in a city without a current store
- Total sales for the competition should be less than \$500,000
- The new store must be located in a city with a population less then 4000 based on teh 2014 census estimate
- Predicted yearly sales must be over \$200,000
- The city chosen should have the hightest predicted sales within the criteria

### What data is needed to make these decisions?
- Demographic data including 2010 and 2014 census data
- Pawdacity sales data for each store in Wyoming
- Competition sales data for each store in Wyoming

### Libraries

In [1]:
library( tidyverse )
library( PerformanceAnalytics )

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last


Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend



### Functions

In [2]:
cleanFun <- function(htmlString) {
  return(gsub("<.*?>", "", htmlString))
}

'%!in%' <- function(x,y)!('%in%'(x,y))

### Data

In [3]:
df.comp <- read.csv( '../data/raw/p2-wy-453910-naics-data.csv' )
df.demo <- read.csv( '../data/raw/p2-wy-demographic-data.csv' )
df.pop <- read.csv( '../data/raw/p2-partially-parsed-wy-web-scrape.csv' )
df.sales <- read.csv( '../data/raw/p2-2010-pawdacity-monthly-sales-p2-2010-pawdacity-monthly-sales.csv' )

### Clean population webscrape data

In [4]:
df.pop <- df.pop[,c(1,3,4)]

In [5]:
names( df.pop ) <- c( 'City.County', 'pop_2010_no', 'pop_2014_est' )

In [6]:
df.pop$pop_2010_no <- cleanFun( df.pop$pop_2010_no )
df.pop$pop_2014_est <- cleanFun( df.pop$pop_2014_est )

In [7]:
df.pop$pop_2010_no <- gsub( '-', '', df.pop$pop_2010_no )
df.pop$pop_2014_est <- gsub( '-', '', df.pop$pop_2014_est )

In [8]:
df.pop$pop_2010_no <- gsub( ',', '', df.pop$pop_2010_no )
df.pop$pop_2014_est <- gsub( ',', '', df.pop$pop_2014_est )

In [9]:
df.pop$City <- substr( df.pop$City.County, 0, gregexpr(pattern ='\\|', df.pop$City.County ) )

In [10]:
df.pop$City <- gsub( '\\|', '', df.pop$City )

In [11]:
df.pop$City <- gsub( '\\?', '', df.pop$City )

In [12]:
df.pop$City <- trimws( df.pop$City, 'right' )

In [13]:
df.pop[df.pop$pop_2010_no == '1285[4]',]$pop_2010_no <- 1285

In [14]:
df.pop[df.pop$pop_2010_no == '366[5]',]$pop_2010_no <- 366

In [15]:
df.pop <- df.pop[,c(4, 2, 3)]

In [16]:
df.pop <- df.pop[df.pop$pop_2010_no != '',]

In [17]:
df.pop$pop_2010_no <- as.numeric( df.pop$pop_2010_no )
df.pop$pop_2014_est <- as.numeric( df.pop$pop_2014_est )

In [18]:
head( df.pop, 10 )

City,pop_2010_no,pop_2014_est
Afton,1911,1818
Albin,181,120
Alpine,828,550
Baggs,440,348
Bairoil,106,97
Bar Nunn,2213,936
Basin,1285,1238
Bear River,518,NA
Big Piney,552,408
Buffalo,4585,3900


In [19]:
df.comb <- df.pop

In [20]:
names( df.comb ) <- c( 'city_nm', 'pop_2010_no', 'pop_2014_est' )

### Split the dataset

#### Select out the training set by filtering for cities with stores

In [21]:
cities <- df.sales$CITY

In [22]:
df.train <- filter( df.comb, df.comb$city_nm %in% cities )

In [23]:
df.train

city_nm,pop_2010_no,pop_2014_est
Buffalo,4585,3900
Casper,35316,32644
Cheyenne,59466,53011
Cody,9520,8835
Douglas,6120,5288
Evanston,12359,11507
Gillette,29087,19646
Powell,6314,5373
Riverton,10615,9310
Rock Springs,23036,18708


In [24]:
df.pred <- filter( df.comb, df.comb$city_nm %!in% cities )

In [25]:
head( df.pred )

city_nm,pop_2010_no,pop_2014_est
Afton,1911,1818
Albin,181,120
Alpine,828,550
Baggs,440,348
Bairoil,106,97
Bar Nunn,2213,936


#### Join sales data for each city

In [26]:
df.sales$city_nm <- df.sales$CITY

In [27]:
df.train <- left_join( df.train, df.sales, by = 'city_nm' )

Warning message:
“Column `city_nm` joining character vector and factor, coercing into character vector”

In [28]:
head( df.train )

city_nm,pop_2010_no,pop_2014_est,NAME,ADDRESS,CITY,STATE,ZIP,January,February,March,April,May,June,July,August,September,October,November,December
Buffalo,4585,3900,Pawdacity,509 Fort St # A,Buffalo,WY,82834,16200,13392,14688,17064,18360,14040,12960,19224,15984,13392,13176,16848
Casper,35316,32644,Pawdacity,601 SE Wyoming Blvd Unit 252,Casper,WY,82609,29160,21600,27000,27648,29160,27216,25488,25704,22896,25272,28944,27648
Cheyenne,59466,53011,Pawdacity,3769 E Lincolnway,Cheyenne,WY,82001,79920,70632,79056,77544,73656,77976,73872,77544,78516,74520,74736,79920
Cody,9520,8835,Pawdacity,2625 Big Horn Ave,Cody,WY,82414,19440,15984,19008,18144,16632,17496,18792,20304,19224,18144,18576,16632
Douglas,6120,5288,Pawdacity,123 S 2nd St,Douglas,WY,82633,16200,13392,14688,17064,18360,14040,12960,19224,15984,29808,17496,18792
Evanston,12359,11507,Pawdacity,932 Main St,Evanston,WY,82930,24840,21168,21600,22248,24192,24624,25488,25704,22032,21168,25920,24840


In [29]:
df.train.sales <- gather( df.train, month, sales
      , January:December )

In [30]:
head( df.train.sales )

city_nm,pop_2010_no,pop_2014_est,NAME,ADDRESS,CITY,STATE,ZIP,month,sales
Buffalo,4585,3900,Pawdacity,509 Fort St # A,Buffalo,WY,82834,January,16200
Casper,35316,32644,Pawdacity,601 SE Wyoming Blvd Unit 252,Casper,WY,82609,January,29160
Cheyenne,59466,53011,Pawdacity,3769 E Lincolnway,Cheyenne,WY,82001,January,79920
Cody,9520,8835,Pawdacity,2625 Big Horn Ave,Cody,WY,82414,January,19440
Douglas,6120,5288,Pawdacity,123 S 2nd St,Douglas,WY,82633,January,16200
Evanston,12359,11507,Pawdacity,932 Main St,Evanston,WY,82930,January,24840


#### Aggregate monthly sales to each city

In [31]:
df.train.sales <- aggregate( df.train.sales$sales
                           , list( df.train.sales$city_nm )
                           , sum )

In [32]:
names( df.train.sales ) <- c( 'city_nm', 'agg.sales' )

In [33]:
df.train <- left_join( df.train, df.train.sales, by = 'city_nm' )

In [34]:
head( df.train )

city_nm,pop_2010_no,pop_2014_est,NAME,ADDRESS,CITY,STATE,ZIP,January,February,⋯,April,May,June,July,August,September,October,November,December,agg.sales
Buffalo,4585,3900,Pawdacity,509 Fort St # A,Buffalo,WY,82834,16200,13392,⋯,17064,18360,14040,12960,19224,15984,13392,13176,16848,185328
Casper,35316,32644,Pawdacity,601 SE Wyoming Blvd Unit 252,Casper,WY,82609,29160,21600,⋯,27648,29160,27216,25488,25704,22896,25272,28944,27648,317736
Cheyenne,59466,53011,Pawdacity,3769 E Lincolnway,Cheyenne,WY,82001,79920,70632,⋯,77544,73656,77976,73872,77544,78516,74520,74736,79920,917892
Cody,9520,8835,Pawdacity,2625 Big Horn Ave,Cody,WY,82414,19440,15984,⋯,18144,16632,17496,18792,20304,19224,18144,18576,16632,218376
Douglas,6120,5288,Pawdacity,123 S 2nd St,Douglas,WY,82633,16200,13392,⋯,17064,18360,14040,12960,19224,15984,29808,17496,18792,208008
Evanston,12359,11507,Pawdacity,932 Main St,Evanston,WY,82930,24840,21168,⋯,22248,24192,24624,25488,25704,22032,21168,25920,24840,283824


In [35]:
df.train <- df.train[,c( 'city_nm', 'pop_2010_no', 'pop_2014_est', 'agg.sales' )]

In [36]:
head( df.train )

city_nm,pop_2010_no,pop_2014_est,agg.sales
Buffalo,4585,3900,185328
Casper,35316,32644,317736
Cheyenne,59466,53011,917892
Cody,9520,8835,218376
Douglas,6120,5288,208008
Evanston,12359,11507,283824


### Add households under 18 numbers

In [37]:
df.demo <- df.demo[,c(1,3,4,5,6)]

In [38]:
head( df.demo )

City,Land.Area,Households.with.Under.18,Population.Density,Total.Families
Laramie,2513.7452,2075,5.19,4668.93
Rock River,200.4440,165,0.41,372.30
Basin,543.9513,250,0.66,566.43
Burlington,137.6462,63,0.17,143.34
Byron,252.4896,116,0.31,262.93
Cowley,297.6807,137,0.36,309.98


In [39]:
df.demo$city_nm <- df.demo$City

In [40]:
df.comb <- left_join( df.comb, df.demo, by = 'city_nm' )

Warning message:
“Column `city_nm` joining character vector and factor, coercing into character vector”

In [41]:
head( df.comb )

city_nm,pop_2010_no,pop_2014_est,City,Land.Area,Households.with.Under.18,Population.Density,Total.Families
Afton,1911,1818,Afton,853.806650,497,0.93,1040.41
Albin,181,120,Albin,7.685902,37,0.10,74.87
Alpine,828,550,Alpine,366.598892,213,0.40,446.72
Baggs,440,348,Baggs,253.240322,62,0.06,129.53
Bairoil,106,97,Bairoil,29.459830,18,0.01,33.70
Bar Nunn,2213,936,Bar Nunn,208.573220,417,0.60,843.88


In [42]:
df.train <- left_join( df.train, df.demo, by = 'city_nm' )

Warning message:
“Column `city_nm` joining character vector and factor, coercing into character vector”

In [43]:
head( df.train)

city_nm,pop_2010_no,pop_2014_est,agg.sales,City,Land.Area,Households.with.Under.18,Population.Density,Total.Families
Buffalo,4585,3900,185328,Buffalo,3115.5075,746,1.55,1819.50
Casper,35316,32644,317736,Casper,3894.3091,7788,11.16,8756.32
Cheyenne,59466,53011,917892,Cheyenne,1500.1784,7158,20.34,14612.64
Cody,9520,8835,218376,Cody,2998.9570,1403,1.82,3515.62
Douglas,6120,5288,208008,Douglas,1829.4651,832,1.46,1744.08
Evanston,12359,11507,283824,Evanston,999.4971,1486,4.95,2712.64


In [44]:
df.train <- df.train[,-5]

In [45]:
head( df.train )

city_nm,pop_2010_no,pop_2014_est,agg.sales,Land.Area,Households.with.Under.18,Population.Density,Total.Families
Buffalo,4585,3900,185328,3115.5075,746,1.55,1819.50
Casper,35316,32644,317736,3894.3091,7788,11.16,8756.32
Cheyenne,59466,53011,917892,1500.1784,7158,20.34,14612.64
Cody,9520,8835,218376,2998.9570,1403,1.82,3515.62
Douglas,6120,5288,208008,1829.4651,832,1.46,1744.08
Evanston,12359,11507,283824,999.4971,1486,4.95,2712.64


In [46]:
names( df.train ) <- c( 'city_nm'
                     , 'pop_2010_no'
                     , 'pop_2014_est'
                     , 'agg_sales_no'
                     , 'land_area_no'
                     , 'households_under_18_no'
                     , 'pop_density_no'
                     , 'total_families_no' )

In [47]:
df.comb <- df.comb[,-4]

In [48]:
head( df.comb )

city_nm,pop_2010_no,pop_2014_est,Land.Area,Households.with.Under.18,Population.Density,Total.Families
Afton,1911,1818,853.806650,497,0.93,1040.41
Albin,181,120,7.685902,37,0.10,74.87
Alpine,828,550,366.598892,213,0.40,446.72
Baggs,440,348,253.240322,62,0.06,129.53
Bairoil,106,97,29.459830,18,0.01,33.70
Bar Nunn,2213,936,208.573220,417,0.60,843.88


In [ ]:
names( df.comb ) <- c( 'city_nm'
                     , 'pop_2010_no'
                     , 'pop_2014_est'
                     , 'land_area_no'
                     , 'households_under_18_no'
                     , 'pop_density_no'
                     , 'total_families_no' )

### Data is claned and split, train regression model

#### Remove Gillete as an outlier

In [ ]:
df.train <- df.train[df.train$city_nm != 'Gillette',]

In [ ]:
df.train

In [ ]:
chart.Correlation( df.train[,-1] )

In [ ]:
chart.Correlation( df.pred[,c(-1, -3)] )